In [3]:
''''TO-DO LIST'''''
''''

1. Qingguo Wei 91% accuracy - http://www.bbci.de/competition/iii/results/tuebingen/QingguoWei_desc.html

Processing technique:

���� Three features are combined to perform classification task. Two of them are extracted by common spatial subspace decomposition (CSSD) and Fisher discriminant analysis (FDA), and another feature is derived from waveform mean (WM) and FDA. The classifier is support vector machines (SVM).


2. Paul Hammon 2nd 87% accuracy - classification SVM but accuracy less

3. Michael Sapinski 3rd 86% accuracy - http://www.bbci.de/competition/iii/results/tuebingen/MichalSapinski_desc.txt

Preprocessing: All channels were downsampled to 25Hz.

Features used: Mean values of selected channels in some periods of time,
mean power of selected frequency intervals for selected channels. These
features were constructed by hand with help of visualization of
differences for the given classes.

Classification: Few logistic regression classifiers that seemed to give
quite similar results on the test set but used different features.


4. Mao Dawei 3rd 86% accuracy - http://www.bbci.de/competition/iii/results/tuebingen/MaoDawei_desc.html

Feature: Data from every electrode was transformed to time-frequency domain by Hilbert-Huang Transform. Extract Standard deviation of every time-frequency window as feature.

Classification method: mahalanobis distance to class center.

Processing Flow:

Hilbert-Huang Transform of ECoG from all electrodes.

Extract Standard deviation of every time-frequency window (5Hz*0.2S) as feature.

Calculated error rate by leave-one-out use train data. Select 7 electrodes (29,30,31,38,39,40,46) that has lowest error rate with single time-frequency window.

Use Standard deviation of time-frequency domain from all 7 electrodes, single frequency band and 11 continuous time windows as feature. Calculated class center of class 1 and calss-1 from train data.

Use mahalanobis distance to classify the test data.

5. Liu Yang 3rd 86% accuracy - http://www.bbci.de/competition/iii/results/tuebingen/LiuYang_desc.txt

Preprocessing:
1) filter signals using bandpass filter(2.5Hz to 25Hz). 
2) get data of 1s length as trials after the position of every cue for both training and testing data.
3) compute CSP(Common Spatial Pattern) using training data, apply it to both training and testing data, then compute cwt of every trials.

Classifying:
1) compute two correlation matrix A(among training trials) and C(between testing trials and training trials)
2) construct the feature space using every row of matrix A and C as features of corresponding trials.
3) optimize a linear discriminant surface from a initial position decided by the training trials under a energy penalty critera, ie.
   min norm(Ax-b)+norm(x) 
where x is weight vector of the linear discriminant function, b is vector containing expected positions of every training samples.

'''

"'\n\n1. Qingguo Wei 91% accuracy - http://www.bbci.de/competition/iii/results/tuebingen/QingguoWei_desc.html\n\nProcessing technique:\n\n���� Three features are combined to perform classification task. Two of them are extracted by common spatial subspace decomposition (CSSD) and Fisher discriminant analysis (FDA), and another feature is derived from waveform mean (WM) and FDA. The classifier is support vector machines (SVM).\n\n\n2. Paul Hammon 2nd 87% accuracy - classification SVM but accuracy less\n\n3. Michael Sapinski 3rd 86% accuracy - http://www.bbci.de/competition/iii/results/tuebingen/MichalSapinski_desc.txt\n\nPreprocessing: All channels were downsampled to 25Hz.\n\nFeatures used: Mean values of selected channels in some periods of time,\nmean power of selected frequency intervals for selected channels. These\nfeatures were constructed by hand with help of visualization of\ndifferences for the given classes.\n\nClassification: Few logistic regression classifiers that seemed t

In [1]:
import numpy as np
from scipy import signal

In [2]:
data=np.load('bci_3.npz')

In [3]:
data_2=data["X"]

In [4]:
#Selecting 8 electrodes as done in the text provided in 2nd best algorithm
data_2_new=data_2[:,(29,30,31,32,37,38,39,40),:]
data_2_new.shape

(278, 8, 3000)

In [5]:
#subsampling by 4 
data_2_sub=data_2_new
data_2_subs=np.zeros((278, 8, 750))
for i in range(0, 278):
    for j in range(0, 8):
        data_2_subs[i, j, :]=signal.resample(data_2_sub[i, j, :], 750)

data_2_subs.shape

(278, 8, 750)

In [6]:
#Common Average Reference
for j in range(0, 278):
    car=np.zeros((750,))
    for i in range(0, 8):
        car= car + data_2_subs[j,i,:]

    car=car/8
    #car.shape

    for k in range(0, 8):
        data_2_subs[j,k,:]=data_2_subs[j,k,:]-car

In [99]:
#Standard Scaler
from sklearn.preprocessing import StandardScaler
for j in range(0, 278):
    kr=data_2_subs[j,:,:]
    
    scaler=StandardScaler().fit(kr.T)
    data_2_subs[j,:,:]=scaler.transform(kr.T).T


In [100]:
''''from sklearn.decomposition import FastICA
ica = FastICA(n_components=8, max_iter=200)

data_ica=np.zeros((278, 8, 750))
for i in range(0, 278):
    data_ica[i, :, :]=ica.fit_transform(data_2_subs[i,:,:].T).T'''


"'from sklearn.decomposition import FastICA\nica = FastICA(n_components=8, max_iter=200)\n\ndata_ica=np.zeros((278, 8, 750))\nfor i in range(0, 278):\n    data_ica[i, :, :]=ica.fit_transform(data_2_subs[i,:,:].T).T"

In [7]:
#bandpass filter
b, a = signal.butter(2, 0.4, 'low', analog=False)
data_2_subs = signal.filtfilt(b, a, data_2_subs, axis=2)

In [8]:
#Extracting all the features and concatenating them 
final = np.array([])
for j in range(0, 278):
    data_trial=data_2_subs[j,:,:].T
    #data_trial.shape

    data_trial_s1=data_trial[0:250,:]
    #print(data_trial_s1.shape)
    data_trial_s2=data_trial[250:500,:]
    #print(data_trial_s2.shape)
    data_trial_s3=data_trial[500:750,:]
    #print(data_trial_s3.shape)

    #AR Coefficients
    import statsmodels.api as sm
    #from statsmodels.datasets.sunspots import load
    #data = load()
    ARFV=np.array([])

    for i in range(0, 8):
        rho1, sigma1 = sm.regression.linear_model.burg(data_trial_s1[:,i], order=2)
        rho2, sigma2 = sm.regression.linear_model.burg(data_trial_s2[:,i], order=2)
        rho3, sigma3 = sm.regression.linear_model.burg(data_trial_s3[:,i], order=2)
        ARFV=np.append(ARFV, (rho1, rho2, rho3))

    #print(ARFV) 

    #Haar wavelet
    import pywt
    HWDFV=np.array([])
    for i in range(0, 8):
        (cA, cD) = pywt.dwt(data_trial[:,i], 'haar')
        HWDFV=np.append(HWDFV, cA)

    #Spectral Power estimates
    SPFV=np.array([])
    for i in range(0, 8):
        f1, Pxx_den1 = signal.welch(data_trial_s1[:,i], 250)
        f2, Pxx_den2 = signal.welch(data_trial_s2[:,i], 250)
        f3, Pxx_den3 = signal.welch(data_trial_s3[:,i], 250)
        SPFV=np.append(SPFV, (Pxx_den1, Pxx_den2, Pxx_den3))

    #Concatenaton of All the feature vectors
    concated=np.concatenate((ARFV, HWDFV, SPFV), axis=None)
    concated=np.reshape(concated, (-1, 1))
    if j==0:
        final=concated
    else:
        final= np.hstack((final, concated))
    print(j)
print(final.shape)

C:\Users\Ritwik\anaconda3\lib\site-packages\scipy\signal\spectral.py:1961: UserWarning: nperseg = 256 is greater than input length  = 250, using nperseg = 250
  warnings.warn('nperseg = {0:d} is greater than input length '


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [9]:
final=final.T
final.shape

(278, 6072)

In [10]:
#Labels of the training data
labels=data['events']
print(labels.shape)
labels=np.reshape(labels, (-1,))
print(labels.shape)

(278, 1)
(278,)


In [11]:
#Create an svm Classifier
from sklearn import svm

clf1 = svm.SVC(kernel='linear') # Linear Kernel

#Train the model using the training sets
clf1.fit(final, labels)

SVC(kernel='linear')

In [12]:
#Doing the same things for the test set

data_2_new_t=data['X_test'][:,(29,30,31,32,37,38,39,40),:]
data_2_new_t.shape

(100, 8, 3000)

In [13]:
#subsampling by 4 
data_2_sub_t=data_2_new_t
data_2_subs_t=np.zeros((100, 8, 750))
for i in range(0, 100):
    for j in range(0, 8):
        data_2_subs_t[i, j, :]=signal.resample(data_2_sub_t[i, j, :], 750)

data_2_subs_t.shape

(100, 8, 750)

In [14]:
#Common Average Reference
for j in range(0, 100):
    car=np.zeros((750,))
    for i in range(0, 8):
        car= car + data_2_subs_t[j,i,:]

    car=car/8
    #car.shape

    for k in range(0, 8):
        data_2_subs_t[j,k,:]=data_2_subs_t[j,k,:]-car

In [109]:
#Standard Scaler
from sklearn.preprocessing import StandardScaler
for j in range(0, 100):
    kr=data_2_subs_t[j,:,:]
    
    scaler=StandardScaler().fit(kr.T)
    data_2_subs_t[j,:,:]=scaler.transform(kr.T).T


In [110]:
#from sklearn.decomposition import FastICA
''''ica = FastICA(n_components=8, max_iter=200)

data_ica_t=np.zeros((100, 8, 750))
for i in range(0, 100):
    data_ica_t[i, :, :]=ica.fit_transform(data_2_subs_t[i,:,:].T).T'''

"'ica = FastICA(n_components=8, max_iter=200)\n\ndata_ica_t=np.zeros((100, 8, 750))\nfor i in range(0, 100):\n    data_ica_t[i, :, :]=ica.fit_transform(data_2_subs_t[i,:,:].T).T"

In [15]:
#bandpass filter
b, a = signal.butter(2, 0.4, 'low', analog=False)
data_2_subs_t = signal.filtfilt(b, a, data_2_subs_t, axis=2)

In [16]:
final_t = np.array([])
for j in range(0 ,100):
    data_trial=data_2_subs_t[j,:,:].T
    #data_trial.shape

    data_trial_s1=data_trial[0:250,:]
    #print(data_trial_s1.shape)
    data_trial_s2=data_trial[250:500,:]
    #print(data_trial_s2.shape)
    data_trial_s3=data_trial[500:750,:]
    #print(data_trial_s3.shape)

    #AR Coefficients
    import statsmodels.api as sm
    #from statsmodels.datasets.sunspots import load
    #data = load()
    ARFV=np.array([])

    for i in range(0, 8):
        rho1, sigma1 = sm.regression.linear_model.burg(data_trial_s1[:,i], order=2)
        rho2, sigma2 = sm.regression.linear_model.burg(data_trial_s2[:,i], order=2)
        rho3, sigma3 = sm.regression.linear_model.burg(data_trial_s3[:,i], order=2)
        ARFV=np.append(ARFV, (rho1, rho2, rho3))

    #print(ARFV) 

    #Haar wavelet
    import pywt
    HWDFV=np.array([])
    for i in range(0, 8):
        (cA, cD) = pywt.dwt(data_trial[:,i], 'haar')
        HWDFV=np.append(HWDFV, cA)

    #Spectral Power estimates
    SPFV=np.array([])
    for i in range(0, 8):
        f1, Pxx_den1 = signal.welch(data_trial_s1[:,i], 250)
        f2, Pxx_den2 = signal.welch(data_trial_s2[:,i], 250)
        f3, Pxx_den3 = signal.welch(data_trial_s3[:,i], 250)
        SPFV=np.append(SPFV, (Pxx_den1, Pxx_den2, Pxx_den3))

    #Concatenaton of All the feature vectors
    concated=np.concatenate((ARFV, HWDFV, SPFV), axis=None)
    concated=np.reshape(concated, (-1, 1))
    if j==0:
        final_t=concated
    else:
        final_t= np.hstack((final_t, concated))
    print(j)
print(final_t.shape)

C:\Users\Ritwik\anaconda3\lib\site-packages\scipy\signal\spectral.py:1961: UserWarning: nperseg = 256 is greater than input length  = 250, using nperseg = 250
  warnings.warn('nperseg = {0:d} is greater than input length '


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
(6072, 100)


In [17]:
final_t=final_t.T
final_t.shape

(100, 6072)

In [18]:
#Loading the true labels for the test set from BCI 3 website
truelabels=np.loadtxt("true_labels.txt", delimiter="/n")

In [19]:
#Predicting the Test labels using previously trained model
y_pred_t = clf1.predict(final_t)

In [20]:
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics

print("Accuracy:",metrics.accuracy_score(truelabels, y_pred_t))

Accuracy: 0.6
